In [10]:
import pandas as pd
import numpy as np
import random
import os
import sys
from math import sqrt
sys.path.append('../..')
from modules import utils
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import tensorflow_probability as tfp
tfk = tf.keras
tf.keras.backend.set_floatx('float64')
tfd = tfp.distributions
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from sklearn.ensemble import IsolationForest
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
%matplotlib inline

In [11]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)
tf.random.set_seed(SEED)

In [12]:
neg_log_likelihood = lambda x, rv_x: -rv_x.log_prob(x)

#### The data

In [13]:
jinja_df = pd.read_csv('../data/jinja_data.csv', parse_dates=['timestamp'])
jinja_df.head()

,site_name,latitude,longitude,city,timestamp,pm2_5_calibrated_value,pm2_5_raw_value,pm10_raw_value,pm10_calibrated_value,site_id,device_number,device_name
0,"Jinja Main Street, Jinja",0.437337,33.211051,Jinja,2021-09-01 00:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6142,689753,aq_23
1,"Jinja Main Street, Jinja",0.437337,33.211051,Jinja,2021-09-01 01:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6142,689753,aq_23
2,"Jinja Main Street, Jinja",0.437337,33.211051,Jinja,2021-09-01 02:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6142,689753,aq_23
3,"Jinja Main Street, Jinja",0.437337,33.211051,Jinja,2021-09-01 03:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6142,689753,aq_23
4,"Jinja Main Street, Jinja",0.437337,33.211051,Jinja,2021-09-01 04:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6142,689753,aq_23


In [14]:
latitudes = jinja_df['latitude'].unique()
longitudes = jinja_df['longitude'].unique()
device_ids = jinja_df['device_number'].unique()
len(latitudes), len(longitudes), len(device_ids)

(10, 10, 10)

In [15]:
final_df = pd.DataFrame()
cols = ['timestamp', 'latitude', 'longitude', 'pm2_5_calibrated_value']
for i, device_id in enumerate(device_ids):
    device_df = utils.get_device_data(jinja_df, device_id, cols)
    processed_df = utils.preprocessing(device_df)
    final_df = pd.concat([final_df, processed_df])
final_df.reset_index(drop=True, inplace=True)
final_df.head()

,time,latitude,longitude,pm2_5
0,452909.0,0.437337,33.211051,12.2844
1,452910.0,0.437337,33.211051,11.6507
2,452911.0,0.437337,33.211051,22.3980
3,452912.0,0.437337,33.211051,17.4937
4,452913.0,0.437337,33.211051,25.1622


#### Model training and validation

In [24]:
def bnn(X_train, y_train, epochs=1000, optimizer='RMSProp', dropout=0.2):
#     model = tf.keras.Sequential([
#     tfp.layers.DenseVariational(units=128, make_prior_fn=tfp.layers.default_multivariate_normal_fn, 
#                                 make_posterior_fn=tfp.layers.default_multivariate_normal_fn),
#     tfp.layers.DenseVariational(units=64, make_prior_fn=tfp.layers.default_multivariate_normal_fn,
#                                make_posterior_fn=tfp.layers.default_multivariate_normal_fn),
#     tfp.layers.DenseVariational(units=1, make_prior_fn=tfp.layers.default_multivariate_normal_fn,
#                                make_posterior_fn=tfp.layers.default_multivariate_normal_fn),
#     ])
    
# #     model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss=tf.keras.losses.MeanSquaredError())
#     model.compile(optimizer=optimizer, loss=tf.keras.losses.MeanSquaredError())
#     model.fit(X_train, y_train, batch_size=32, epochs=epochs, callbacks=None, validation_split=0.2)
    prior = tfd.Independent(tfd.Normal(loc=tf.zeros(1, dtype=tf.float64), scale=1.0), 
                            reinterpreted_batch_ndims=1)
    model = tfk.Sequential([
        tfk.layers.InputLayer(input_shape=(3,), name='input'),
        tfk.layers.Dropout(dropout, name='dropout1'),
        tfk.layers.Dense(10, activation='relu', name='dense_1'),
        tfk.layers.Dropout(dropout, name='dropout2'),
        tfk.layers.Dense(tfp.layers.MultivariateNormalTriL.params_size(1), activation=None, name='distribution_weights'),
        tfk.layers.Dropout(dropout, name='dropout3'),
        tfp.layers.MultivariateNormalTriL(1, activity_regularizer=tfp.layers.KLDivergenceRegularizer(prior, weight=1/32), 
                                          name='output')], name='model')
    
#     model.compile(optimizer=optimizer, loss=neg_log_likelihood)
    model.compile(optimizer=optimizer, loss=tf.keras.losses.MeanSquaredError())
    checkpoint = ModelCheckpoint('../models/bnn_checkpoint.h5', monitor='val_loss', save_best_only=True,
                                save_weights_only=False)
    early_stopping = EarlyStopping(monitor='val_loss', patience=300)
    model.fit(X_train, y_train, batch_size=32, epochs=epochs, callbacks=[checkpoint, early_stopping], 
              validation_split=0.2)
    return model
#     best_model = keras.models.load_model('../models/bnn_checkpoint.h5')
#     return best_model

In [17]:
def cross_validation(final_df, idx):
    device_indices = final_df[final_df.latitude==latitudes[idx]].index
    device_df = jinja_df[jinja_df.device_number == device_ids[idx]]
    assert(len(device_indices) == len(device_df)-device_df.pm2_5_calibrated_value.isna().sum())
    
    test_df = final_df.loc[device_indices]
    assert(len(test_df.longitude.unique()) == 1)
    
    train_df = pd.concat([final_df, test_df]).drop_duplicates(keep=False)
    assert(len(train_df.longitude.unique()) == len(longitudes)-1)
    assert len(final_df) == len(test_df) + len(train_df)
    
    
    X_train = train_df.iloc[:, 0:-1]
    y_train = train_df.iloc[:, -1]
#     data_train =tf.data.Dataset.from_tensor_slices((X_train.values, y_train.values))
#     data_train = data_train.batch(32).repeat(n_epochs)
    X_train, y_train = np.array(X_train), np.array(y_train)#.reshape(-1, 1)
    
    X_test = test_df.iloc[:, 0:-1]
    y_test = test_df.iloc[:, -1]
#     data_test =tf.data.Dataset.from_tensor_slices((X_test.values, y_test.values))
#     data_test = data_test.batch(1)
    X_test, y_test = np.array(X_test), np.array(y_test)#.reshape(-1, 1)
    
    model = bnn(X_train, y_train)
    y_pred = model.predict(X_test)
    
    rmse = sqrt(mean_squared_error(y_test, y_pred))
    mape = mean_absolute_percentage_error(y_test, y_pred)
    return rmse, mape

#### delete from here

In [25]:
idx=0
device_indices = final_df[final_df.latitude==latitudes[idx]].index
device_df = jinja_df[jinja_df.device_number == device_ids[idx]]
test_df = final_df.loc[device_indices]
train_df = pd.concat([final_df, test_df]).drop_duplicates(keep=False)
X_train = train_df.iloc[:, 0:-1]
y_train = train_df.iloc[:, -1]
X_train, y_train = np.array(X_train), np.array(y_train)
X_test = test_df.iloc[:, 0:-1]
y_test = test_df.iloc[:, -1]
X_test, y_test = np.array(X_test), np.array(y_test)

In [26]:
# %%timeit
model = bnn(X_train, y_train)

Epoch 1/1000
289/289 [==============================] - 2s 3ms/step - loss: 23150675834.1922 - val_loss: 15034235.1290
Epoch 2/1000
289/289 [==============================] - 0s 2ms/step - loss: 2566340362.9704 - val_loss: 950218.6764
Epoch 3/1000
289/289 [==============================] - 0s 2ms/step - loss: 60840228.3673 - val_loss: 2630.2390
Epoch 4/1000
289/289 [==============================] - 1s 2ms/step - loss: 1470.7844 - val_loss: 1393.7035
Epoch 5/1000
289/289 [==============================] - 0s 1ms/step - loss: 664.2573 - val_loss: 955.1997
Epoch 6/1000
289/289 [==============================] - 0s 2ms/step - loss: 651.7958 - val_loss: 888.8868
Epoch 7/1000
289/289 [==============================] - 0s 1ms/step - loss: 647.9975 - val_loss: 945.7890
Epoch 8/1000
289/289 [==============================] - 0s 1ms/step - loss: 641.9986 - val_loss: 828.5535
Epoch 9/1000
289/289 [==============================] - 0s 1ms/step - loss: 648.6950 - val_loss: 823.1371
Epoch 10/1000
2

289/289 [==============================] - 0s 2ms/step - loss: 564.5149 - val_loss: 850.5783
Epoch 78/1000
289/289 [==============================] - 0s 2ms/step - loss: 565.6703 - val_loss: 716.1409
Epoch 79/1000
289/289 [==============================] - 0s 1ms/step - loss: 562.2711 - val_loss: 910.4567
Epoch 80/1000
289/289 [==============================] - 0s 1ms/step - loss: 558.3824 - val_loss: 930.2507
Epoch 81/1000
289/289 [==============================] - 0s 1ms/step - loss: 565.9509 - val_loss: 913.7003
Epoch 82/1000
289/289 [==============================] - 0s 2ms/step - loss: 561.2533 - val_loss: 727.9376
Epoch 83/1000
289/289 [==============================] - 0s 2ms/step - loss: 566.4099 - val_loss: 782.4806
Epoch 84/1000
289/289 [==============================] - 0s 1ms/step - loss: 556.7622 - val_loss: 926.2731
Epoch 85/1000
289/289 [==============================] - 0s 2ms/step - loss: 558.1598 - val_loss: 849.4215
Epoch 86/1000
289/289 [============================

289/289 [==============================] - 0s 1ms/step - loss: 543.7518 - val_loss: 789.4327
Epoch 154/1000
289/289 [==============================] - 0s 1ms/step - loss: 543.6037 - val_loss: 789.7547
Epoch 155/1000
289/289 [==============================] - 0s 1ms/step - loss: 541.5229 - val_loss: 789.0330
Epoch 156/1000
289/289 [==============================] - 0s 1ms/step - loss: 547.8978 - val_loss: 788.0954
Epoch 157/1000
289/289 [==============================] - 0s 1ms/step - loss: 537.4114 - val_loss: 787.5894
Epoch 158/1000
289/289 [==============================] - 0s 2ms/step - loss: 538.6546 - val_loss: 786.5670
Epoch 159/1000
289/289 [==============================] - 0s 1ms/step - loss: 544.8479 - val_loss: 787.4711
Epoch 160/1000
289/289 [==============================] - 0s 1ms/step - loss: 540.3838 - val_loss: 786.6199
Epoch 161/1000
289/289 [==============================] - 0s 2ms/step - loss: 550.4218 - val_loss: 786.9343
Epoch 162/1000
289/289 [===================

Epoch 229/1000
289/289 [==============================] - 0s 1ms/step - loss: 541.5221 - val_loss: 781.4911
Epoch 230/1000
289/289 [==============================] - 0s 1ms/step - loss: 555.6196 - val_loss: 782.0206
Epoch 231/1000
289/289 [==============================] - 0s 2ms/step - loss: 541.3764 - val_loss: 781.8563
Epoch 232/1000
289/289 [==============================] - 0s 2ms/step - loss: 545.7112 - val_loss: 782.4754
Epoch 233/1000
289/289 [==============================] - 0s 2ms/step - loss: 544.9583 - val_loss: 781.9171
Epoch 234/1000
289/289 [==============================] - 0s 1ms/step - loss: 547.1895 - val_loss: 782.3191
Epoch 235/1000
289/289 [==============================] - 0s 1ms/step - loss: 546.2245 - val_loss: 782.4280
Epoch 236/1000
289/289 [==============================] - 0s 2ms/step - loss: 541.6912 - val_loss: 781.9285
Epoch 237/1000
289/289 [==============================] - 0s 2ms/step - loss: 542.9588 - val_loss: 782.1497
Epoch 238/1000
289/289 [====

Epoch 305/1000
289/289 [==============================] - 0s 1ms/step - loss: 534.6490 - val_loss: 781.1939
Epoch 306/1000
289/289 [==============================] - 0s 1ms/step - loss: 540.8916 - val_loss: 781.4791
Epoch 307/1000
289/289 [==============================] - 0s 1ms/step - loss: 547.3828 - val_loss: 781.2490
Epoch 308/1000
289/289 [==============================] - 0s 2ms/step - loss: 552.5958 - val_loss: 781.1771
Epoch 309/1000
289/289 [==============================] - 0s 1ms/step - loss: 542.0369 - val_loss: 780.6565
Epoch 310/1000
289/289 [==============================] - 0s 2ms/step - loss: 539.8792 - val_loss: 780.5547
Epoch 311/1000
289/289 [==============================] - 0s 2ms/step - loss: 540.1977 - val_loss: 780.8266
Epoch 312/1000
289/289 [==============================] - 0s 2ms/step - loss: 537.2262 - val_loss: 780.6155
Epoch 313/1000
289/289 [==============================] - 0s 1ms/step - loss: 536.8642 - val_loss: 780.4048
Epoch 314/1000
289/289 [====

Epoch 381/1000
289/289 [==============================] - 0s 2ms/step - loss: 539.3122 - val_loss: 780.8723
Epoch 382/1000
289/289 [==============================] - 0s 2ms/step - loss: 540.8275 - val_loss: 780.7116
Epoch 383/1000
289/289 [==============================] - 0s 1ms/step - loss: 546.1285 - val_loss: 780.9538
Epoch 384/1000
289/289 [==============================] - 0s 1ms/step - loss: 539.9171 - val_loss: 780.9336
Epoch 385/1000
289/289 [==============================] - 0s 2ms/step - loss: 540.6517 - val_loss: 781.3000
Epoch 386/1000
289/289 [==============================] - 0s 1ms/step - loss: 544.0011 - val_loss: 781.4248
Epoch 387/1000
289/289 [==============================] - 0s 1ms/step - loss: 538.6473 - val_loss: 780.7982
Epoch 388/1000
289/289 [==============================] - 0s 1ms/step - loss: 545.2702 - val_loss: 781.5473
Epoch 389/1000
289/289 [==============================] - 0s 1ms/step - loss: 546.9712 - val_loss: 781.4736
Epoch 390/1000
289/289 [====

In [27]:
%%timeit
y_pred = model.predict(X_test)

66/66 [==============================] - 0s 694us/step
86 ms ± 5.56 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### end here

In [9]:
rmse_list, mape_list = [], []
for i in range(len(latitudes)):
    rmse, mape = cross_validation(final_df, i)
    rmse_list.append(rmse)
    mape_list.append(mape)
    print(f'{device_ids[i]} successful')

2023-07-13 15:12:20.044293: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-07-13 15:12:20.044324: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: PL1207-PRO.paris.inria.fr
2023-07-13 15:12:20.044327: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: PL1207-PRO.paris.inria.fr
2023-07-13 15:12:20.044461: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 525.125.6
2023-07-13 15:12:20.044492: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: NOT_FOUND: could not find kernel module information in driver version file contents: "NVRM version: NVIDIA UNIX Open Kernel Module for x86_64  525.125.06  Release Build  (dvs-builder@U16-A23-14-1)  Tue May 30 05:12:48 UTC 2023
GCC version:  gcc ve

Epoch 1/1000
289/289 [==============================] - 2s 2ms/step - loss: 3323009467386.2769 - val_loss: 4650766268.5502
Epoch 2/1000
289/289 [==============================] - 0s 1ms/step - loss: 984942153710.5459 - val_loss: 355651315.8257
Epoch 3/1000
289/289 [==============================] - 0s 1ms/step - loss: 168252595186.0685 - val_loss: 14154727.9042
Epoch 4/1000
289/289 [==============================] - 0s 1ms/step - loss: 15317030181.6245 - val_loss: 27816.8938
Epoch 5/1000
289/289 [==============================] - 0s 1ms/step - loss: 80373515.9062 - val_loss: 9504.0503
Epoch 6/1000
289/289 [==============================] - 0s 1ms/step - loss: 1090460.5966 - val_loss: 1658.2860
Epoch 7/1000
289/289 [==============================] - 0s 1ms/step - loss: 640341.4147 - val_loss: 1525.6671
Epoch 8/1000
289/289 [==============================] - 0s 1ms/step - loss: 244610.4804 - val_loss: 657.5228
Epoch 9/1000
289/289 [==============================] - 0s 1ms/step - loss: 46

289/289 [==============================] - 0s 1ms/step - loss: 897.2150 - val_loss: 1310.2221
Epoch 152/1000
289/289 [==============================] - 0s 1ms/step - loss: 878.1652 - val_loss: 1311.4431
Epoch 153/1000
289/289 [==============================] - 0s 1ms/step - loss: 884.5461 - val_loss: 1308.7329
Epoch 154/1000
289/289 [==============================] - 0s 1ms/step - loss: 886.1186 - val_loss: 1310.8737
Epoch 155/1000
289/289 [==============================] - 0s 1ms/step - loss: 879.8998 - val_loss: 1309.5774
Epoch 156/1000
289/289 [==============================] - 0s 1ms/step - loss: 887.5558 - val_loss: 1309.2471
Epoch 157/1000
289/289 [==============================] - 0s 1ms/step - loss: 885.5276 - val_loss: 1308.8428
Epoch 158/1000
289/289 [==============================] - 0s 1ms/step - loss: 889.2680 - val_loss: 1308.6723
Epoch 159/1000
289/289 [==============================] - 0s 1ms/step - loss: 885.6485 - val_loss: 1311.8798
Epoch 160/1000
289/289 [==========

289/289 [==============================] - 1s 2ms/step - loss: 887.0750 - val_loss: 1311.9833
Epoch 302/1000
289/289 [==============================] - 1s 2ms/step - loss: 882.5152 - val_loss: 1310.6262
Epoch 303/1000
289/289 [==============================] - 1s 2ms/step - loss: 893.1096 - val_loss: 1312.5332
Epoch 304/1000
289/289 [==============================] - 1s 2ms/step - loss: 881.6909 - val_loss: 1312.8543
Epoch 305/1000
289/289 [==============================] - 1s 2ms/step - loss: 879.9716 - val_loss: 1309.9486
Epoch 306/1000
289/289 [==============================] - 1s 2ms/step - loss: 884.2314 - val_loss: 1310.5450
Epoch 307/1000
289/289 [==============================] - 1s 2ms/step - loss: 888.1147 - val_loss: 1310.0396
Epoch 308/1000
289/289 [==============================] - 1s 2ms/step - loss: 893.8201 - val_loss: 1311.6456
Epoch 309/1000
289/289 [==============================] - 1s 2ms/step - loss: 888.0623 - val_loss: 1310.0318
Epoch 310/1000
289/289 [==========

289/289 [==============================] - 0s 1ms/step - loss: 884.1670 - val_loss: 1311.6821
Epoch 452/1000
289/289 [==============================] - 0s 1ms/step - loss: 887.5940 - val_loss: 1313.0262
Epoch 453/1000
289/289 [==============================] - 0s 1ms/step - loss: 887.3590 - val_loss: 1311.9836
Epoch 454/1000
289/289 [==============================] - 0s 1ms/step - loss: 887.6690 - val_loss: 1313.2896
Epoch 455/1000
289/289 [==============================] - 0s 1ms/step - loss: 885.9345 - val_loss: 1312.1507
Epoch 456/1000
289/289 [==============================] - 0s 2ms/step - loss: 882.2966 - val_loss: 1311.8943
Epoch 457/1000
289/289 [==============================] - 0s 2ms/step - loss: 893.2197 - val_loss: 1312.9799
Epoch 458/1000
289/289 [==============================] - 0s 2ms/step - loss: 886.9779 - val_loss: 1313.4295
Epoch 459/1000
289/289 [==============================] - 0s 2ms/step - loss: 888.5738 - val_loss: 1313.5615
Epoch 460/1000
289/289 [==========

...

Epoch 683/1000
315/315 [==============================] - 0s 1ms/step - loss: 831.3256 - val_loss: 779.4036
Epoch 684/1000
315/315 [==============================] - 0s 1ms/step - loss: 829.9639 - val_loss: 777.8299
Epoch 685/1000
315/315 [==============================] - 0s 1ms/step - loss: 828.4342 - val_loss: 778.6814
Epoch 686/1000
315/315 [==============================] - 0s 1ms/step - loss: 824.7860 - val_loss: 776.1787
Epoch 687/1000
315/315 [==============================] - 0s 1ms/step - loss: 821.7564 - val_loss: 776.2116
Epoch 688/1000
315/315 [==============================] - 0s 1ms/step - loss: 826.1531 - val_loss: 777.6348
Epoch 689/1000
315/315 [==============================] - 0s 1ms/step - loss: 835.7204 - val_loss: 777.7905
Epoch 690/1000
315/315 [==============================] - 0s 1ms/step - loss: 834.7741 - val_loss: 778.6531
Epoch 691/1000
315/315 [==============================] - 0s 1ms/step - loss: 828.0218 - val_loss: 777.6713
Epoch 692/1000
315/315 [====

Epoch 759/1000
315/315 [==============================] - 0s 1ms/step - loss: 839.1167 - val_loss: 779.7894
Epoch 760/1000
315/315 [==============================] - 0s 1ms/step - loss: 829.7203 - val_loss: 779.9262
Epoch 761/1000
315/315 [==============================] - 0s 1ms/step - loss: 828.2598 - val_loss: 779.3241
Epoch 762/1000
315/315 [==============================] - 0s 1ms/step - loss: 827.1533 - val_loss: 778.6391
Epoch 763/1000
315/315 [==============================] - 0s 1ms/step - loss: 831.3781 - val_loss: 779.0262
Epoch 764/1000
315/315 [==============================] - 0s 1ms/step - loss: 827.9244 - val_loss: 779.0389
Epoch 765/1000
315/315 [==============================] - 0s 1ms/step - loss: 833.3725 - val_loss: 779.6409
Epoch 766/1000
315/315 [==============================] - 0s 1ms/step - loss: 828.7565 - val_loss: 777.7620
Epoch 767/1000
315/315 [==============================] - 0s 1ms/step - loss: 829.2059 - val_loss: 777.8011
Epoch 768/1000
315/315 [====

Epoch 835/1000
315/315 [==============================] - 0s 1ms/step - loss: 822.5886 - val_loss: 776.9465
Epoch 836/1000
315/315 [==============================] - 0s 1ms/step - loss: 828.4838 - val_loss: 779.1420
Epoch 837/1000
315/315 [==============================] - 0s 1ms/step - loss: 834.5872 - val_loss: 778.1610
Epoch 838/1000
315/315 [==============================] - 0s 1ms/step - loss: 822.5862 - val_loss: 778.0799
Epoch 839/1000
315/315 [==============================] - 0s 1ms/step - loss: 834.4285 - val_loss: 778.8953
Epoch 840/1000
315/315 [==============================] - 0s 1ms/step - loss: 822.2041 - val_loss: 778.0901
Epoch 841/1000
315/315 [==============================] - 0s 1ms/step - loss: 834.6272 - val_loss: 778.0440
Epoch 842/1000
315/315 [==============================] - 0s 1ms/step - loss: 831.5610 - val_loss: 778.9985
Epoch 843/1000
315/315 [==============================] - 0s 1ms/step - loss: 826.2650 - val_loss: 777.3343
Epoch 844/1000
315/315 [====

Epoch 911/1000
315/315 [==============================] - 1s 4ms/step - loss: 827.0760 - val_loss: 778.6631
Epoch 912/1000
315/315 [==============================] - 1s 4ms/step - loss: 828.8140 - val_loss: 777.6950
Epoch 913/1000
315/315 [==============================] - 1s 4ms/step - loss: 822.1437 - val_loss: 775.6969
Epoch 914/1000
315/315 [==============================] - 1s 4ms/step - loss: 837.4314 - val_loss: 777.7239
Epoch 915/1000
315/315 [==============================] - 1s 4ms/step - loss: 829.5917 - val_loss: 778.0297
Epoch 916/1000
315/315 [==============================] - 1s 4ms/step - loss: 833.6040 - val_loss: 776.9519
Epoch 917/1000
315/315 [==============================] - 1s 2ms/step - loss: 826.8227 - val_loss: 778.3047
Epoch 918/1000
315/315 [==============================] - 0s 1ms/step - loss: 833.6669 - val_loss: 777.6365
Epoch 919/1000
315/315 [==============================] - 0s 1ms/step - loss: 826.9319 - val_loss: 777.4538
Epoch 920/1000
315/315 [====

Epoch 987/1000
315/315 [==============================] - 0s 1ms/step - loss: 829.9394 - val_loss: 776.8680
Epoch 988/1000
315/315 [==============================] - 0s 1ms/step - loss: 825.8753 - val_loss: 776.8610
Epoch 989/1000
315/315 [==============================] - 0s 1ms/step - loss: 828.2079 - val_loss: 778.0190
Epoch 990/1000
315/315 [==============================] - 0s 1ms/step - loss: 837.5652 - val_loss: 778.1286
Epoch 991/1000
315/315 [==============================] - 0s 1ms/step - loss: 833.6468 - val_loss: 778.5859
Epoch 992/1000
315/315 [==============================] - 0s 1ms/step - loss: 824.0291 - val_loss: 778.8650
Epoch 993/1000
315/315 [==============================] - 0s 1ms/step - loss: 830.3683 - val_loss: 778.7457
Epoch 994/1000
315/315 [==============================] - 0s 1ms/step - loss: 830.6376 - val_loss: 777.7044
Epoch 995/1000
315/315 [==============================] - 0s 1ms/step - loss: 829.8864 - val_loss: 778.2230
Epoch 996/1000
315/315 [====

In [10]:
mean_rmse = np.mean(rmse_list)          
mean_rmse

28.317615833730308

In [11]:
rmse_list

[16.20393648098763,
 37.54040791891388,
 23.43678751759825,
 31.639449089521936,
 18.351870625481293,
 34.39396625654075,
 28.16110720754395,
 18.831284654206694,
 36.54474974086948,
 38.07259884563923]

In [12]:
mean_mape = np.mean(mape_list)
mean_mape

0.7457903690497681

In [13]:
mape_list

[0.6831006969325372,
 0.790365907273025,
 0.7211546589885162,
 0.7905894817929953,
 0.7087969353764905,
 0.7685189906871869,
 0.7506637968947788,
 0.6829209546099062,
 0.7769657290645953,
 0.7848265388776505]